In [41]:
from sklearn.model_selection import KFold
from IPython.display import display
from sklearn.neural_network import MLPClassifier

In [2]:
# Data manipulation
import pandas as pd
import numpy as np


# Splitting data
from sklearn.model_selection import train_test_split

N_FOLDS = 5
MAX_EVALS = 5

In [61]:
anto = pd.read_table("../antonym-synonym set/Antonym_vietnamese.txt", sep=" ", header=None)
syn = pd.read_table("../antonym-synonym set/Synonym_vietnamese.txt", sep=" ", header=None)
anto["label"] = 0
syn["label"] = 1
display(syn.describe())
display(anto.describe())
data = pd.concat([syn, anto])
data.columns = ["first", "second", "label"]
display(data.describe())

,label
count,11562.0
mean,1.0
std,0.0
min,1.0
25%,1.0
50%,1.0
75%,1.0
max,1.0


,label
count,2000.0
mean,0.0
std,0.0
min,0.0
25%,0.0
50%,0.0
75%,0.0
max,0.0


,label
count,13562.000000
mean,0.852529
std,0.354588
min,0.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,1.000000


In [47]:
from scipy import stats
from scipy import spatial
from gensim.models import KeyedVectors


In [55]:
word2vec = KeyedVectors.load_word2vec_format('../word2vec/W2V_150.txt', binary=False)

[-1.096853    0.1238883   1.713642    2.17519    -0.5785002  -2.027245
  0.4998034  -0.7409065  -1.753515   -1.095152    0.7456962   1.484329
  1.235524    1.025137   -0.4033328  -0.9089296  -0.2022127   0.9581299
  0.6957457   1.385916    0.4481485   1.238998    0.1075599   3.29651
  1.315556    0.4587597   2.277924   -1.313732    0.9624746  -0.2428055
 -0.9468833   0.8783497   0.3211554  -0.5890836   0.9027938  -1.178614
 -0.7399899  -1.011717    1.766519   -2.162154    0.9139196  -2.295288
 -1.206698   -2.222441   -1.830276   -0.4457759   1.122905    0.3227312
 -0.03411892 -0.7967375   0.1875879   0.3892174  -0.7192345  -0.5432259
 -0.6546717  -0.7268866   0.5100322   0.9201084   0.2751797   1.194452
  0.5092697  -0.3654743  -1.434594   -1.484043    1.105507    0.02227577
 -1.812047   -1.204937   -3.369251    0.6383803   0.1732161  -0.478294
 -0.5762202   0.1084443  -0.1128882   0.647718   -0.3459211   0.5320156
  0.239954   -2.538636   -0.7471865   1.441165    0.02344817 -0.9431902

In [70]:
print(word2vec['a_dua'])
import random
print(np.random.randn(150))

[-1.096853    0.1238883   1.713642    2.17519    -0.5785002  -2.027245
  0.4998034  -0.7409065  -1.753515   -1.095152    0.7456962   1.484329
  1.235524    1.025137   -0.4033328  -0.9089296  -0.2022127   0.9581299
  0.6957457   1.385916    0.4481485   1.238998    0.1075599   3.29651
  1.315556    0.4587597   2.277924   -1.313732    0.9624746  -0.2428055
 -0.9468833   0.8783497   0.3211554  -0.5890836   0.9027938  -1.178614
 -0.7399899  -1.011717    1.766519   -2.162154    0.9139196  -2.295288
 -1.206698   -2.222441   -1.830276   -0.4457759   1.122905    0.3227312
 -0.03411892 -0.7967375   0.1875879   0.3892174  -0.7192345  -0.5432259
 -0.6546717  -0.7268866   0.5100322   0.9201084   0.2751797   1.194452
  0.5092697  -0.3654743  -1.434594   -1.484043    1.105507    0.02227577
 -1.812047   -1.204937   -3.369251    0.6383803   0.1732161  -0.478294
 -0.5762202   0.1084443  -0.1128882   0.647718   -0.3459211   0.5320156
  0.239954   -2.538636   -0.7471865   1.441165    0.02344817 -0.9431902

In [147]:
def get_emb(s):
    if s in word2vec:
        return word2vec[s]
    else:
        return np.random.randn(150)
# Extract the labels
labels = np.array(data['label'].astype(np.int32)).reshape((-1, ))
features = data.drop(columns = ['label'])
print(features)
features['first_emb'] = features.apply(lambda row: get_emb(row['first']), axis=1)
features['second_emb'] = features.apply(lambda row: get_emb(row['second']), axis=1)
features['cos'] = features.apply(lambda row: spatial.distance.cosine(row['first_emb'], row['second_emb']), axis=1)
features['norm_of_diff'] = features.apply(lambda row: np.linalg.norm(row['first_emb'] - row['second_emb']), axis=1)
features = features.drop(columns = ['first', 'second', 'first_emb', 'second_emb'])
print(labels)
print(features)
X = features
y = labels

         first     second
0        a_dua     a_tòng
1        a_dua    vào_hùa
2      a_ma_tơ     tài_tử
3       a_tòng    vào_hùa
4         à_ơi       ạ_ơi
...        ...        ...
1995  bẩn_thỉu    sạch_sẽ
1996       bận       rảnh
1997       bận  rảnh_rang
1998       bận   rảnh_rỗi
1999       bận        rỗi

[13562 rows x 2 columns]
[1 1 1 ... 0 0 0]
           cos  norm_of_diff
0     1.060939     18.984823
1     0.711136     14.071492
2     1.025458     18.777913
3     1.013381     15.411342
4     1.143855     14.500365
...        ...           ...
1995  0.616982     15.439382
1996  0.528708     18.200443
1997  0.553031     16.851196
1998  0.520573     18.138960
1999  0.559956     19.350819

[13562 rows x 2 columns]


In [180]:
from sklearn.linear_model import *
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score
kf = KFold(n_splits=5, shuffle=True)

cnt = 1
# split()  method generate indices to split data into training and test set.
for train_index, test_index in kf.split(X, y):
    print(f'Fold:{cnt}, Train set: {len(train_index)}, Test set:{len(test_index)}')
    cnt += 1
    
result = cross_val_score(LogisticRegression(), X, y, scoring="f1", cv = kf)
print(result)
print("average F1-score: ", result.mean())

result = cross_val_score(MLPClassifier(hidden_layer_sizes=(2, 1), 
                                       solver='adam', 
                                       learning_rate='invscaling', 
                                       learning_rate_init=0.1,
                                      max_iter=1000), X, y, scoring="f1", cv = kf)
print(result)
print("average F1-score: ", result.mean())

Fold:1, Train set: 10849, Test set:2713
Fold:2, Train set: 10849, Test set:2713
Fold:3, Train set: 10850, Test set:2712
Fold:4, Train set: 10850, Test set:2712
Fold:5, Train set: 10850, Test set:2712
[0.92062861 0.92105786 0.91887582 0.91866029 0.92209857]
average F1-score:  0.9202642296719864
[0.91696607 0.91869271 0.92102646 0.92444973 0.92081178]
average F1-score:  0.9203893484611866
